# Training a Torch Classifier

This tutorial demonstrates how to train an image classifier using the [Ray AI Runtime](air) (AIR).

You should be familiar with [PyTorch](https://pytorch.org/) before starting the tutorial. If you need a refresher, read PyTorch's [training a classifier](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) tutorial.

## Before you begin

* Install the [Ray AI Runtime](air). You'll need Ray 1.13 later to run this example.

In [1]:
!pip install 'ray[air]'

* Install `requests`, `torch`, and `torchvision`

In [2]:
!pip install requests torch torchvision

## Load and normalize CIFAR-10

We'll train our classifier on a popular image dataset called [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html).

First, let's load CIFAR-10 into a Ray Dataset.

In [1]:
import ray
from ray.data.datasource import SimpleTorchDatasource
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

def train_dataset_factory():
    return torchvision.datasets.CIFAR10(root="./data", download=True, train=True, transform=transform)

def test_dataset_factory():
    return torchvision.datasets.CIFAR10(root="./data", download=True, train=False, transform=transform)

train_dataset: ray.data.Dataset = ray.data.read_datasource(SimpleTorchDatasource(), dataset_factory=train_dataset_factory)
test_dataset: ray.data.Dataset = ray.data.read_datasource(SimpleTorchDatasource(), dataset_factory=test_dataset_factory)

2022-08-02 22:04:35,185	INFO worker.py:1481 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265.
2022-08-02 22:04:36,120	WARNING read_api.py:291 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
(_get_read_tasks pid=56761) 2022-08-02 22:04:36,118	WARNING torch_datasource.py:55 -- `SimpleTorchDatasource` doesn't support parallel reads. The `parallelism` argument will be ignored.


(_execute_read_task pid=56761) Files already downloaded and verified


2022-08-02 22:04:41,132	WARNING read_api.py:291 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
(_get_read_tasks pid=56761) 2022-08-02 22:04:41,129	WARNING torch_datasource.py:55 -- `SimpleTorchDatasource` doesn't support parallel reads. The `parallelism` argument will be ignored.


(_execute_read_task pid=56761) Files already downloaded and verified


In [2]:
train_dataset

Dataset(num_blocks=1, num_rows=50000, schema=<class 'tuple'>)

Note that {py:class}`SimpleTorchDatasource <ray.data.datasource.SimpleTorchDatasource>` loads all data into memory, so you shouldn't use it with larger datasets.

Next, let's represent our data using pandas dataframes instead of tuples. This lets us call methods like {py:meth}`Dataset.to_torch <ray.data.Dataset.to_torch>` later in the tutorial.

In [3]:
from typing import Tuple
import pandas as pd
from ray.data.extensions import TensorArray
import torch


def convert_batch_to_pandas(batch: Tuple[torch.Tensor, int]) -> pd.DataFrame:
    images = TensorArray([image.numpy() for image, _ in batch])
    labels = [label for _, label in batch]

    df = pd.DataFrame({"image": images, "label": labels})

    return df


train_dataset = train_dataset.map_batches(convert_batch_to_pandas)
test_dataset = test_dataset.map_batches(convert_batch_to_pandas)

Read->Map_Batches:   0%|                                                                                                                                                                                              | 0/1 [00:00<?, ?it/s]

(_map_block_nosplit pid=56761) Files already downloaded and verified


Read->Map_Batches:   0%|                                                                                                                                                                                              | 0/1 [00:00<?, ?it/s]

(_map_block_nosplit pid=56761) Files already downloaded and verified


Read->Map_Batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.32s/it]


In [4]:
train_dataset

Dataset(num_blocks=1, num_rows=50000, schema={image: TensorDtype(shape=(3, 32, 32), dtype=float32), label: int64})

## Train a convolutional neural network

Now that we've created our datasets, let's define the training logic.

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

We define our training logic in a function called `train_loop_per_worker`.

`train_loop_per_worker` contains regular PyTorch code with a few notable exceptions:
* We wrap our model with {py:func}`train.torch.prepare_model <ray.train.torch.prepare_model>`.
* We call {py:func}`session.get_dataset_shard <ray.air.session.get_dataset_shard>` and {py:meth}`Dataset.to_torch <ray.data.Dataset.to_torch>` to convert a subset of our training data to a Torch dataset.
* We save model state using {py:func}`session.report <ray.air.session.report>`.

In [6]:
from ray import train
from ray.air import session, Checkpoint
import torch.optim as optim


def train_loop_per_worker(config):
    model = train.torch.prepare_model(Net())

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    train_dataset_shard: torch.utils.data.Dataset = session.get_dataset_shard("train").to_torch(
        feature_columns=["image"],
        label_column="label",
        batch_size=config["batch_size"],
        unsqueeze_feature_tensors=False,
        unsqueeze_label_tensor=False
    )

    for epoch in range(2):
        running_loss = 0.0
        for i, data in enumerate(train_dataset_shard):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            print(inputs)
            print(labels)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}")
                running_loss = 0.0

        session.report(
            dict(running_loss=running_loss),
            checkpoint=Checkpoint.from_dict(dict(model=model.module.state_dict())),
        )

Finally, we can train our model. This should take a few minutes to run.

In [7]:
from ray.train.torch import TorchTrainer
from ray.air.config import ScalingConfig

trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config={"batch_size": 2},
    datasets={"train": train_dataset},
    scaling_config=ScalingConfig(num_workers=2)
)
result = trainer.fit()
latest_checkpoint = result.checkpoint

Trial name,status,loc,iter,total time (s),running_loss,_timestamp,_time_this_iter_s
TorchTrainer_ffdc5_00000,RUNNING,127.0.0.1:56948,1,47.6922,681.787,1659503223,44.3013


(RayTrainWorker pid=56966) 2022-08-02 22:06:17,139	INFO config.py:71 -- Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pid=56967) [W ProcessGroupGloo.cpp:715] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
(RayTrainWorker pid=56966) [W ProcessGroupGloo.cpp:715] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
(RayTrainWorker pid=56966) 2022-08-02 22:06:19,392	INFO train_loop_utils.py:300 -- Moving model to device: cpu
(RayTrainWorker pid=56966) 2022-08-02 22:06:19,392	INFO train_loop_utils.py:347 -- Wrapping provided model in DDP.
(RayTrainWorker pid=56967) /Users/cindyz/mambaforge/envs/ray/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
(Ray

To scale your training script, create a [Ray Cluster](deployment-guide) and increase the number of workers. If your cluster contains GPUs, add `"use_gpu": True` to your scaling config.

```{code-block} python
scaling_config=ScalingConfig(num_workers=8, "use_gpu=True)
```

## Test the network on the test data

Let's see how our model performs.

To classify images in the test dataset, we'll need to create a {py:class}`Predictor <ray.train.predictor.Predictor>`.

{py:class}`Predictors <ray.train.predictor.Predictor>` load data from checkpoints and efficiently perform inference. In contrast to {py:class}`TorchPredictor <ray.train.torch.TorchPredictor>`, which performs inference on a single batch, {py:class}`BatchPredictor <ray.train.batch_predictor.BatchPredictor>` performs inference on an entire dataset. Because we want to classify all of the images in the test dataset, we'll use a {py:class}`BatchPredictor <ray.train.batch_predictor.BatchPredictor>`.

In [8]:
from ray.train.torch import TorchPredictor
from ray.train.batch_predictor import BatchPredictor

predict_dataset = test_dataset.drop_columns(cols=["label"])
batch_predictor = BatchPredictor.from_checkpoint(
    checkpoint=latest_checkpoint,
    predictor_cls=TorchPredictor,
    model=Net(),
)

outputs: ray.data.Dataset = batch_predictor.predict(
    data=test_dataset, dtype=torch.float, feature_columns=["image"], keep_columns=["label"]
)

Map Progress (1 actors 1 pending):   0%|                                                                                                                                                                              | 0/1 [00:00<?, ?it/s](BlockWorker pid=82548) /Users/cindyz/mambaforge/envs/ray/lib/python3.8/site-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
(BlockWorker pid=82548) A value is trying to be set on a copy of a slice from a DataFrame.
(BlockWorker pid=82548) Try using .loc[row_indexer,col_indexer] = value instead
(BlockWorker pid=82548) 
(BlockWorker pid=82548) See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
(BlockWorker pid=82548)   self.obj[selected_item_labels] = value
Map Progress (1 actors 1 pending): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

Our model outputs a list of energies for each class. To classify an image, we
choose the class that has the highest energy.

In [9]:
import numpy as np

def convert_logits_to_classes(df):
    best_class = df["predictions"].map(lambda x: x.argmax())
    df["prediction"] = best_class
    return df

predictions = outputs.map_batches(
    convert_logits_to_classes, batch_format="pandas"
)

predictions.show(1)

Map_Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 25.02it/s]

{'predictions': array([-0.06768818, -3.0893242 ,  1.4991069 ,  2.4655461 ,  0.02666938,
        2.176965  ,  0.45571244, -0.18882632, -0.74345344, -2.676772  ],
      dtype=float32), 'label': 3, 'prediction': 3}


Now that we've classified all of the images, let's figure out which images were
classified correctly. The ``predictions`` dataset contains predicted labels and 
the ``test_dataset`` contains the true labels. To determine whether an image 
was classified correctly, we join the two datasets and check if the predicted 
labels are the same as the actual labels.

In [10]:
def calculate_prediction_scores(df):
    df["correct"] = df["prediction"] == df["label"]
    return df[["prediction", "label", "correct"]]

scores = predictions.map_batches(calculate_prediction_scores)

scores.show(1)

Map_Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 38.97it/s]

{'prediction': 3, 'label': 3, 'correct': True}


To compute our test accuracy, we'll count how many images the model classified 
correctly and divide that number by the total number of test images.

In [11]:
scores.sum(on="correct") / scores.count()

Shuffle Reduce: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 65.69it/s]


0.5638

## Deploy the network and make a prediction

Our model seems to perform decently, so let's deploy the model to an 
endpoint. This'll allow us to make predictions over the Internet.

In [13]:
from ray import serve
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import NdArray

def json_to_numpy(payload: NdArray) -> pd.DataFrame:
      """Accepts an NdArray JSON from an HTTP body and converts it to a Numpy Array."""
      # Have to explicitly convert to float since np.array reads as a double.
      arr = np.array(payload.array, dtype=np.float32)
      return arr

deployment = PredictorDeployment.options(name="my-deployment")
serve.run(
    deployment.bind(
        TorchPredictor, 
        latest_checkpoint, 
        batching_params=False, 
        model=Net(), 
        http_adapter=json_to_numpy
    )
)

(ServeController pid=82655) INFO 2022-08-02 11:20:57,850 controller 82655 http_state.py:123 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-0699de2a8ddbd4136a0c5a1df4c4570896df0a089c72e707af1a4408' on node '0699de2a8ddbd4136a0c5a1df4c4570896df0a089c72e707af1a4408' listening on '127.0.0.1:8000'
(ServeController pid=82655) INFO 2022-08-02 11:20:58,369 controller 82655 deployment_state.py:1232 - Adding 1 replicas to deployment 'my-deployment'.
(HTTPProxyActor pid=82658) INFO:     Started server process [82658]
You are retrieving a sync handle inside an asyncio loop. Try getting Deployment.get_handle(.., sync=False) to get better performance. Learn more at https://docs.ray.io/en/latest/serve/handle-guide.html#sync-and-async-handles


RayServeSyncHandle(deployment='my-deployment')

Let's classify a test image.

In [14]:
batch = test_dataset.take(1)
array = np.expand_dims(np.array(batch[0]["image"]), axis=0)

In [15]:
array.shape

(1, 3, 32, 32)

You can perform inference against a deployed model by posting a dictionary with an `"array"` key. To learn more about the default input schema, read the {py:class}`NdArray <ray.serve.http_adapters.NdArray>` documentation.

In [16]:
import requests

payload = {"array": array.tolist()}
response = requests.post("http://localhost:8000/", json=payload)
response.json()

[[-0.06768877804279327,
  -3.089324474334717,
  1.499107003211975,
  2.4655468463897705,
  0.0266684889793396,
  2.1769652366638184,
  0.45571354031562805,
  -0.18882650136947632,
  -0.7434539198875427,
  -2.676771640777588]]

(HTTPProxyActor pid=82658) INFO 2022-08-02 11:21:04,553 http_proxy 127.0.0.1 http_proxy.py:315 - POST /predict 307 15.4ms
(HTTPProxyActor pid=82658) INFO 2022-08-02 11:21:04,573 http_proxy 127.0.0.1 http_proxy.py:315 - POST /predict 200 18.1ms
(ServeReplica:my-deployment pid=82660) INFO 2022-08-02 11:21:04,547 my-deployment my-deployment#mFmsUf replica.py:482 - HANDLE __call__ OK 0.7ms
(ServeReplica:my-deployment pid=82660) INFO 2022-08-02 11:21:04,570 my-deployment my-deployment#mFmsUf replica.py:482 - HANDLE __call__ OK 9.7ms
